
----
# Cuaderno 7 - Clasificación con KNN (K-Nearest Neighbors)
## Ariel Palazzesi - 2026
----

En este cuaderno vamos a aplicar el algoritmo **KNN** (K-Nearest Neighbors) para resolver un problema de clasificación binaria utilizando el conjunto de datos del Titanic.

Nuestro objetivo será predecir si una persona **sobrevivió** o no (`Survived`) a partir de variables como edad, clase del pasaje, sexo, y cantidad de familiares a bordo.

Ya conocés este dataset por las clases anteriores, pero en esta oportunidad aplicaremos un nuevo modelo que **no entrena una fórmula**, sino que **toma decisiones observando los vecinos más cercanos** a cada observación.

Además de entrenar el modelo, evaluaremos su rendimiento y lo compararemos con el obtenido previamente con regresión logística.


## K-Nearest Neighbors (KNN) y clasificación binaria

Comencemos repasando algunos conceptos.

El algoritmo **K-Nearest Neighbors (KNN)** es un método de **aprendizaje supervisado** que se utiliza tanto para problemas de **clasificación** como de regresión. En el caso de clasificación binaria, su objetivo es asignar a cada observación una de dos clases posibles en función de la similitud con los datos ya conocidos.

A diferencia de otros modelos, KNN es un algoritmo **no paramétrico y basado en instancias**. Esto significa que no aprende un modelo matemático durante una etapa de entrenamiento tradicional. En su lugar, **almacena los datos de entrenamiento** y, cuando debe clasificar una nueva observación, busca los *K* vecinos más cercanos dentro del conjunto de datos y decide la clase por votación mayoritaria.

La cercanía entre observaciones se mide generalmente utilizando la **distancia euclidiana**, aunque existen otras métricas posibles. La elección del valor de *K* es un aspecto clave del algoritmo:

* valores pequeños de *K* hacen al modelo más sensible al ruido,
* valores grandes suavizan la decisión, pero pueden perder detalles importantes.

En scikit-learn, KNN para clasificación se implementa mediante la clase `KNeighborsClassifier`, que sigue la misma interfaz que otros modelos: se ajusta con `fit()` y se realizan predicciones con `predict()`.

---

### Funcionamiento general del algoritmo

Cuando se quiere clasificar una nueva observación, KNN:

1. calcula la distancia entre esa observación y todos los datos de entrenamiento,
2. selecciona los *K* vecinos más cercanos,
3. analiza qué clase aparece con mayor frecuencia entre esos vecinos,
4. asigna esa clase como predicción final.

Este enfoque hace que KNN sea un algoritmo **intuitivo y fácil de entender**, ya que se basa directamente en la idea de similitud entre datos.

---

### Métricas de evaluación en clasificación

Al tratarse de un problema de clasificación, el rendimiento de KNN se evalúa con las mismas métricas que otros modelos clasificadores:

* La **precisión (accuracy)** indica qué proporción de las predicciones totales fue correcta.

* El **recall** mide qué proporción de los casos positivos reales fue correctamente identificada por el modelo.

* El **precision** indica qué proporción de las predicciones positivas realizadas por el modelo fue correcta.

* El **F1-score** combina precision y recall en una sola métrica, ofreciendo una medida equilibrada del desempeño del modelo.

Sabemos que estas métricas se obtienen fácilmente en scikit-learn utilizando `classification_report()`, que presenta un resumen claro del rendimiento del modelo para cada clase.

---

### Consideraciones importantes sobre KNN

KNN suele funcionar bien como **modelo base** o como punto de comparación, pero presenta algunas características a tener en cuenta:

* puede ser costoso computacionalmente cuando el conjunto de datos es grande,
* es sensible a la escala de las variables, por lo que suele requerir normalización,
* su rendimiento depende fuertemente de la elección del valor de *K*.

A pesar de estas limitaciones, KNN es un algoritmo muy valioso desde el punto de vista didáctico, ya que permite comprender de forma clara cómo la **similitud entre datos** puede utilizarse para resolver problemas de clasificación.

¡Comencemos!

## Carga del dataset

En este cuaderno seguimos trabajando con el archivo `Titanic-Dataset.csv`, que ya deberías tener cargado en tu entorno de Google Colab.

En el siguiente bloque vamos a importar las librerías necesarias y cargar el archivo. Luego verificaremos que los datos se hayan cargado correctamente mostrando las primeras filas.


In [ ]:
# Importamos las librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importamos herramientas de Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Configuramos estilo de gráficos
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

# Cargamos el dataset desde la carpeta de trabajo
ruta = "/content/Titanic-Dataset.csv"
df = pd.read_csv(ruta)

# Mostramos las primeras filas para verificar la carga
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preparación del dataset para KNN

Como el algoritmo KNN calcula distancias entre puntos, necesitamos seleccionar variables **numéricas y comparables**. Vamos a trabajar con:

- `Pclass`: clase del pasajero (1, 2 o 3)
- `Sex`: codificado como 0 (male) y 1 (female)
- `Age`: edad
- `SibSp`: cantidad de hermanos o cónyuges a bordo
- `Parch`: cantidad de padres o hijos a bordo

También es importante recordar que **KNN es muy sensible a las escalas** de las variables. Por eso, después de seleccionar las variables y eliminar los valores faltantes, vamos a escalar los datos para que todas las columnas influyan por igual en el cálculo de distancias.


In [ ]:
# Hacemos una copia del dataset original
df_knn = df.copy()

# Codificamos la variable 'Sex' (male = 0, female = 1)
df_knn['Sex_encoded'] = df_knn['Sex'].map({'male': 0, 'female': 1})

# Seleccionamos las columnas a utilizar
columnas = ['Survived', 'Pclass', 'Sex_encoded', 'Age', 'SibSp', 'Parch']
df_knn = df_knn[columnas]

# Eliminamos filas con valores faltantes
df_knn = df_knn.dropna()

# Separamos variables predictoras y variable objetivo
X = df_knn[['Pclass', 'Sex_encoded', 'Age', 'SibSp', 'Parch']]
y = df_knn['Survived']

# Escalamos las variables numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Mostramos los primeros datos escalados
pd.DataFrame(X_scaled, columns=X.columns).head()


,Pclass,Sex_encoded,Age,SibSp,Parch
0,0.911232,-0.759051,-0.530377,0.524570,-0.505895
1,-1.476364,1.317434,0.571831,0.524570,-0.505895
2,0.911232,1.317434,-0.254825,-0.551703,-0.505895
3,-1.476364,1.317434,0.365167,0.524570,-0.505895
4,0.911232,-0.759051,0.365167,-0.551703,-0.505895


## División del dataset y entrenamiento con K = 3

Vamos a dividir los datos en dos conjuntos:

- **80%** para entrenamiento (X_train, y_train)
- **20%** para prueba (X_test, y_test)

Después, entrenaremos un modelo de **KNN** usando `k = 3`. Esto significa que para cada nueva observación, el modelo observará los **3 vecinos más cercanos** del conjunto de entrenamiento y decidirá su clase según el voto de la mayoría.


In [ ]:
# Dividimos el dataset escalado en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Creamos el modelo con k = 3 vecinos
knn_3 = KNeighborsClassifier(n_neighbors=3)

# Entrenamos el modelo
knn_3.fit(X_train, y_train)

# Generamos predicciones sobre el conjunto de prueba
y_pred_3 = knn_3.predict(X_test)

# Mostramos resultados preliminares
print("Predicciones (primeras 10):", y_pred_3[:10])


Predicciones (primeras 10): [0 1 1 1 0 1 0 0 1 1]


## Primeras predicciones del modelo (k = 3)

El modelo KNN con `k = 3` ya está entrenado y realizamos las primeras predicciones sobre el conjunto de prueba. A modo ilustrativo, las primeras 10 predicciones fueron:

`[0 1 1 1 0 1 0 0 1 1]`

Esto significa que, para cada una de esas observaciones, el modelo observó sus 3 vecinos más cercanos (en el conjunto de entrenamiento) y votó cuál era la clase más común. Por ejemplo, si entre los 3 vecinos había 2 que no sobrevivieron y 1 que sí, el modelo predijo “0”.

Este resultado **no nos dice todavía qué tan bien o mal está funcionando el modelo** —para eso necesitamos compararlo contra los valores reales y calcular métricas como precisión, recall, F1-score, y matriz de confusión— pero ya nos permite ver que el modelo está asignando ambas clases (`0` y `1`), lo cual es un primer indicador de que está operativo.

Ahora vamos a evaluarlo formalmente.


## Evaluación del modelo KNN con k = 3

Vamos a evaluar el rendimiento del modelo utilizando dos herramientas clave:

- **`classification_report`**: muestra precisión, recall y F1-score para cada clase, junto con métricas promedio.
- **`confusion_matrix`**: nos permite ver de forma detallada cuántos casos acertó el modelo y en qué tipo de errores incurrió.

Estas métricas son fundamentales para comparar este modelo con el de regresión logística que entrenamos en clases anteriores.


In [ ]:
# Importamos nuevamente las funciones por si no están cargadas
from sklearn.metrics import classification_report, confusion_matrix

# Evaluamos el modelo
print("Reporte de clasificación para K = 3:\n")
print(classification_report(y_test, y_pred_3))

print("Matriz de confusión:\n")
print(confusion_matrix(y_test, y_pred_3))


Reporte de clasificación para K = 3:

              precision    recall  f1-score   support

           0       0.80      0.76      0.78        87
           1       0.66      0.71      0.68        56

    accuracy                           0.74       143
   macro avg       0.73      0.74      0.73       143
weighted avg       0.75      0.74      0.74       143

Matriz de confusión:

[[66 21]
 [16 40]]


## Interpretación de los resultados para k = 3

El modelo KNN con `k = 3` obtuvo una **precisión general (accuracy)** del **74%**, lo cual indica que acierta en aproximadamente 3 de cada 4 casos del conjunto de prueba.

Si miramos las métricas por clase:

- Para la clase `0` (no sobrevivió), el modelo tiene una **precisión de 0.80** y un **recall de 0.76**, lo que indica que se desempeña bien en detectar correctamente a quienes no sobrevivieron.
- Para la clase `1` (sí sobrevivió), la **precisión baja a 0.66**, pero el **recall es 0.71**, lo cual sugiere que el modelo detecta relativamente bien a los sobrevivientes, aunque comete más errores al clasificarlos.

La **matriz de confusión** muestra:
- **66 verdaderos negativos** (predijo 0 y era 0).
- **21 falsos positivos** (predijo 1, pero era 0).
- **40 verdaderos positivos** (predijo 1 y era 1).
- **16 falsos negativos** (predijo 0, pero era 1).

Este modelo tiene un rendimiento razonablemente equilibrado, aunque ligeramente mejor en detectar a quienes no sobrevivieron. Vamos a compararlo con otro valor de `k` para ver si se puede mejorar la precisión o la estabilidad de las predicciones.


## Entrenamiento del modelo con k = 7

Ahora vamos a crear un nuevo modelo KNN, pero esta vez utilizando **7 vecinos más cercanos** en lugar de 3. El objetivo es comparar su rendimiento con el modelo anterior y observar si mejora o empeora la clasificación.

Recordá que valores más altos de `k` suelen dar lugar a modelos más estables, pero también pueden suavizar demasiado la decisión y perder detalles locales importantes.


In [ ]:
# Creamos un nuevo modelo con k = 7 vecinos
knn_7 = KNeighborsClassifier(n_neighbors=7)

# Entrenamos el modelo
knn_7.fit(X_train, y_train)

# Hacemos predicciones sobre el conjunto de prueba
y_pred_7 = knn_7.predict(X_test)

# Evaluamos el modelo
print("Reporte de clasificación para K = 7:\n")
print(classification_report(y_test, y_pred_7))

print("Matriz de confusión:\n")
print(confusion_matrix(y_test, y_pred_7))


Reporte de clasificación para K = 7:

              precision    recall  f1-score   support

           0       0.82      0.86      0.84        87
           1       0.77      0.71      0.74        56

    accuracy                           0.80       143
   macro avg       0.80      0.79      0.79       143
weighted avg       0.80      0.80      0.80       143

Matriz de confusión:

[[75 12]
 [16 40]]


## Comparación de resultados: K = 3 vs K = 7

El modelo KNN con `k = 7` mostró una **mejora clara en el rendimiento general**, alcanzando una precisión (accuracy) del **80%**, frente al 74% que habíamos obtenido con `k = 3`.

Al analizar las métricas por clase:

- Para la clase `0` (no sobrevivió), la precisión subió a **0.82** y el recall a **0.86**, lo que indica que el modelo mejora aún más su capacidad de detectar correctamente a quienes no sobrevivieron.
- Para la clase `1` (sobrevivió), la precisión también mejora: **0.77** frente al 0.66 anterior, aunque el recall se mantiene constante en **0.71**.

La **matriz de confusión** para `k = 7` muestra:
- **75 verdaderos negativos** (predijo 0 y era 0).
- **12 falsos positivos** (predijo 1, pero era 0).
- **40 verdaderos positivos** (predijo 1 y era 1).
- **16 falsos negativos** (predijo 0, pero era 1).

La reducción de falsos positivos y el aumento en la precisión general indican que `k = 7` es una **mejor elección en este caso**, logrando un modelo más equilibrado y con mejor capacidad de generalización.

Esto demuestra cómo el valor de `k` afecta directamente el comportamiento del modelo, y por qué es importante probar varios valores para encontrar el más adecuado según los datos.


## Conclusión y cierre

En este cuaderno aplicamos el algoritmo **K-Nearest Neighbors (KNN)** para resolver un problema de clasificación binaria utilizando el dataset del Titanic. Probamos con diferentes valores de `k` (3 y 7), analizamos sus efectos en el rendimiento del modelo y reflexionamos sobre cómo las decisiones se ven influenciadas por la cantidad de vecinos considerados.

Observamos que **k = 7 ofreció un mejor equilibrio**, logrando mayor precisión general y menos errores de clasificación. También vimos que, a diferencia de otros modelos, KNN no ajusta una fórmula sino que **observa los datos para tomar decisiones**, lo que lo convierte en una herramienta poderosa pero sensible a la escala, la cantidad de datos y el valor de `k`.

Este enfoque puede ser útil en ciertos escenarios, pero puede ser necesario algo mas: un modelo que pueda **aprender reglas claras a partir de los datos, estructurar decisiones y visualizar caminos alternativos**.

Ese será el desafío que encararemos en el próximo cuaderno: construir modelos usando **árboles de decisión**, una herramienta que transforma datos en ramas lógicas fáciles de seguir, analizar y explicar.